In [1]:
import pandas as pd
# Load CSV file
csv_file = "data/datacleaned.csv"
data = pd.read_csv(csv_file)
data.head()

,Summary,Action,Data,Expected Result
0,Audio Output Selection_Audio management after ...,Go to *Device Preferences >> Display & Sound >...,NaN,Audio is well heard from HDMI\nAudio is well h...
1,"Audio Output Selection_Speaker test _Play ""Spe...","Open Youtube and search for video ""Speaker Hea...",Example Video ID: 6TWJaFD6R2s,Check that the sound is heard on the left side...
2,Audio Output Selection_Check that the Switch b...,From Settings menu >Display & Sound > Audio pr...,NaN,"1. Switch is OK and Audio Video are OK, \n\nSy..."
3,Audio Output Selection_Local speaker,"From Settings Menu, select local speakers as A...",NaN,Verify that audio is correctly outputed on the...
4,Audio Ouput Selection_Check Audio selection af...,"From Settings Menu, select HDMI as Audio outpu...",NaN,"The audio is heared from the TV, and is well s..."


In [2]:
texts = data['Summary '].tolist()
actions = data["Action"].tolist()
expected_results = data["Expected Result"].tolist()

In [3]:
metadata = [
    {"action": actions[i], "expected_result": expected_results[i]}
    for i in range(len(texts))
]

In [4]:
from langchain_nomic.embeddings import NomicEmbeddings
embedding = NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode="local")
embeddings = embedding.embed(texts, task_type="search_query")

Failed to load libllamamodel-mainline-cuda-avxonly.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory
Failed to load libllamamodel-mainline-cuda.so: dlopen: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [5]:
import chromadb
client = chromadb.Client()
collection = client.create_collection(name="Test_Cases")
for i, text in enumerate(texts):
    collection.add(
        ids=[str(i)],
        embeddings=[embeddings[i]],
        documents=[text],  
        metadatas=[metadata[i]],
    )

In [6]:
collection

Collection(id=85e15547-7eb2-4423-acea-e2e774b15a4b, name=Test_Cases)

In [7]:
def view_collection(collection):
    data = collection.get()
    print("Collection Contents:")
    for i, (doc, meta) in enumerate(zip(data["documents"], data["metadatas"])):
        print(f"Entry {i + 1}:")
        print("Document (Summary):", doc)
        print("Metadata:", meta)
        print("-" * 50)
view_collection(collection)


Collection Contents:
Entry 1:
Document (Summary): Audio Output Selection_Audio management after electrical reboot in both modes (HDMI and Speaker)
Metadata: {'action': 'Go to *Device Preferences >> Display & Sound >> Audio processing >> enable Output to HDMI*\n\nCheck the behaviour of the audio after electrical when the audio output is set to HDMI\nGo to *Device Preferences >> Display & Sound >> Audio processing >> disable Output to HDMI*\n\nCheck the behaviour of the audio after electrical wehn the audio output is set to speaker', 'expected_result': 'Audio is well heard from HDMI\nAudio is well heard from speaker'}
--------------------------------------------------
Entry 2:
Document (Summary): Audio Output Selection_Speaker test _Play "Speaker Headphones, Test Left and Right" video
Metadata: {'action': 'Open Youtube and search for video "Speaker Headphones, Test Left and Right" then \xa0launch video', 'expected_result': 'Check that the sound is heard on the left side then on the right

In [8]:
groq_api_key="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"

In [9]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama-3.1-70b-versatile"
)

In [10]:
def query_with_llm(collection, query, embedding, llm, n_results=3):
    query_embedding = embedding.embed([query], task_type="search_query")[0]
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    for i, result in enumerate(results["documents"]):
        document = result
        metadata = results["metadatas"][i]  
        if isinstance(metadata, list):
            metadata = metadata[0]
        if isinstance(metadata, dict):
            prompt = f"""
            Query: {query}

            Document Summary: {document}

            Action to take: {metadata.get('action', 'No action found')}

            Expected Result: {metadata.get('expected_result', 'No expected result found')}

            Based on the above information, generate a concise and informative summary or suggestion.
            """
            response = llm.invoke(prompt)
            print(f"\n{'='*80}")
            print(f"Result {i + 1}:")
            print(f"{'='*80}")
            print(f"\n📄 Summary:")
            print(f"{'='*40}")
            print(f"{document}\n")
            print(f"💡 Action to Take:")
            print(f"{'='*40}")
            print(f"{metadata.get('action', 'No action found')}\n")
            print(f"✅ Expected Result:")
            print(f"{'='*40}")
            print(f"{metadata.get('expected_result', 'No expected result found')}\n")
            print(f"🤖 LLM Generated Result:")
            print(f"{'='*40}")
            print(f"{response}\n")
            print(f"{'='*80}")
        else:
            print(f"Metadata for result {i + 1} is not in the correct format: {metadata}")
            print(f"{'='*80}")

In [11]:
query = "Endurance_Customer Journey"
results = query_with_llm(collection, query, embedding, llm, n_results=1)


Result 1:

📄 Summary:
['Endurance_Customer Journey ']

💡 Action to Take:
Launch following customer journey script on VSB in loop during 24h
Press Menu
Netflix:

1.Go to Netflix
2. In the Netflix Home Page, navigate on the catalog
3. In Netflix Home Page, make a search for a movie
4. Play a Netflix movie during 1 minute, without user interaction
5. Fast forward, go to 10 min
6. Play the movie during 5 min, without user interaction
7. Change the volume with RCU (V+/V-)
8. Do some trick modes
9. Change volume with OK Google (V+/V-)
10. Press exit key and go back to Live
Live + timeshift:

1. Press P+ and wait for the Live
2. Play Live during 5 min
3. Change volume with RCU (V+/V-)
4. Backward trickmodes then play the Live in timeshift during 5 min
5. Change volume with OK Google (V+/V-)
6. Forward trickmodes then play Live in direct during 1 min
VOD

1.Go to VOD and navigate into the VOD menu
2. Play a VOD during 1 min, without user interaction
3. Fast forward, go to 10 min
4. Play the m